<a href="https://colab.research.google.com/github/ortegatiago/covid19_analysis/blob/master/data_capture_by_johns_hopkins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import requests

jonhs_hopkins_github_url = 'https://api.github.com/repos/CSSEGISandData/COVID-19/contents/csse_covid_19_data/csse_covid_19_daily_reports'

In [9]:
response = requests.get(jonhs_hopkins_github_url)

if response:
    github_content_files = response.json()


In [11]:
    for file in github_content_files:
        if '.csv' in file['download_url']:
            print(file['name'])  
            #download(file['name'],file['download_url'], tmp_folder)

01-22-2020.csv
01-23-2020.csv
01-24-2020.csv
01-25-2020.csv
01-26-2020.csv
01-27-2020.csv
01-28-2020.csv
01-29-2020.csv
01-30-2020.csv
01-31-2020.csv
02-01-2020.csv
02-02-2020.csv
02-03-2020.csv
02-04-2020.csv
02-05-2020.csv
02-06-2020.csv
02-07-2020.csv
02-08-2020.csv
02-09-2020.csv
02-10-2020.csv
02-11-2020.csv
02-12-2020.csv
02-13-2020.csv
02-14-2020.csv
02-15-2020.csv
02-16-2020.csv
02-17-2020.csv
02-18-2020.csv
02-19-2020.csv
02-20-2020.csv
02-21-2020.csv
02-22-2020.csv
02-23-2020.csv
02-24-2020.csv
02-25-2020.csv
02-26-2020.csv
02-27-2020.csv
02-28-2020.csv
02-29-2020.csv
03-01-2020.csv
03-02-2020.csv
03-03-2020.csv
03-04-2020.csv
03-05-2020.csv
03-06-2020.csv
03-07-2020.csv
03-08-2020.csv
03-09-2020.csv
03-10-2020.csv
03-11-2020.csv
03-12-2020.csv
03-13-2020.csv
03-14-2020.csv
03-15-2020.csv
03-16-2020.csv
03-17-2020.csv
03-18-2020.csv
03-19-2020.csv
03-20-2020.csv
03-21-2020.csv
03-22-2020.csv
03-23-2020.csv
03-24-2020.csv
03-25-2020.csv
03-26-2020.csv
03-27-2020.csv
03-28-2020